In [ ]:
## 미리 알려줄 것!! python의 exec 용법!

In [1]:
import requests
import json
import pandas as pd
import hashlib # hash 함수용 sha256 사용할 라이브러리|
import random

## 노드의 블록 정보 확인

In [3]:
headers = {'Content-Type' : 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/chain", headers=headers)
json.loads(res.content)

{'chain': [{'index': 1,
   'nonce': 100,
   'previous_hash': 1,
   'timestamp': 1673518970.3987138,
   'transactions': []}],
 'length': 1}

## transaction 입력하기

In [4]:
headers = {'Content-Type' : 'application/json; charset=utf-8'}
data = {
        "sender": "test_from",
        "recipient": "smart_contract",
        "amount": 0,
        "smart_contract": {"contract_address":"myaddress",
                           "contract_code":"print('hello smart contract')"}
}
requests.post("http://localhost:5000/transactions/new", headers=headers, data=json.dumps(data)).content

b'{"message":"Transaction will be added to Block {2}"}\n'

## 노드의 블록정보 확인 - 2

In [5]:
headers = {'Content-Type' : 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/chain", headers=headers)
json.loads(res.content)

{'chain': [{'index': 1,
   'nonce': 100,
   'previous_hash': 1,
   'timestamp': 1673518970.3987138,
   'transactions': []}],
 'length': 1}

##  채굴하기 - 1

In [6]:
headers = {'Content-Type' : 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/mine")
print(res)

<Response [200]>


## 노드의 블록정보 확인 - 3

In [7]:
headers = {'Content-Type' : 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/chain", headers=headers)
json.loads(res.content)

{'chain': [{'index': 1,
   'nonce': 100,
   'previous_hash': 1,
   'timestamp': 1673518970.3987138,
   'transactions': []},
  {'index': 2,
   'nonce': 680492,
   'previous_hash': 'd2d23c670f90f314481441fb226c8a9018008773ec95d84d891d5ae69e52d87d',
   'timestamp': 1673518979.840789,
   'transactions': [{'amount': 0,
     'recipient': 'smart_contract',
     'sender': 'test_from',
     'smart_contract': {'contract_address': 'myaddress',
      'contract_code': "print('hello smart contract')"},
     'timestamp': 1673518977.002757},
    {'amount': 0.1,
     'recipient': 'node_5000',
     'sender': 'master',
     'smart_contract': {'contract_address': 'mining_profit'},
     'timestamp': 1673518979.840789}]}],
 'length': 2}

## Smart Contract 호출 -1

In [8]:
headers = {'Content-Type' : 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/chain", headers=headers)
res_json = json.loads(res.content)

In [10]:
for _block in res_json['chain']:
    for _tx in _block['transactions']:
        if _tx['smart_contract']['contract_address'] == 'myaddress':
            exec( _tx['smart_contract']['contract_code'])
            

hello smart contract


## Smart Contract 입력하기 -2  / 계산기

In [11]:
headers = {'Content-Type' : 'application/json; charset=utf-8'}
data = {
        "sender": "test_from",
        "recipient": "smart_contract",
        "amount": 0,
        "smart_contract": {"contract_address":"calculator",
                           "contract_code" : "calculate_result = {}{}{}"}
}
requests.post("http://localhost:5000/transactions/new", headers=headers, data=json.dumps(data)).content

headers = {'Content-Type' : 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/mine")
print(res)

<Response [200]>


In [12]:
headers = {'Content-Type' : 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/chain", headers=headers)
res_json = json.loads(res.content)

for _block in res_json['chain']:
    for _tx in _block['transactions']:
        if _tx['smart_contract']['contract_address'] == 'calculator':
            exec( _tx['smart_contract']['contract_code'].format(3,"*",5))
            break
print(calculate_result)        

15


## Smart Contract 입력하기 -3  / Token 만들기

In [13]:
headers = {'Content-Type' : 'application/json; charset=utf-8'}
data = {
        "sender": "test_from",
        "recipient": "smart_contract",
        "amount": 0,
        "smart_contract": {"contract_address":"make_token",
                           "contract_code" :"token_name = 'pyTOKEN' \ntoken_total_volume = 10000"
                                            }
            }
requests.post("http://localhost:5000/transactions/new", headers=headers, data=json.dumps(data)).content

headers = {'Content-Type' : 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/mine")
print(res)

<Response [200]>


In [14]:
headers = {'Content-Type' : 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/chain", headers=headers)
res_json = json.loads(res.content)

for _block in res_json['chain']:
    for _tx in _block['transactions']:
        if _tx['smart_contract']['contract_address'] == 'make_token':
            exec( _tx['smart_contract']['contract_code'])
            break
    

In [15]:
token_name

'pyTOKEN'

In [16]:
token_total_volume

10000

## Smart Contract 입력하기 -4  / 거래 가능한 Token 만들기

In [17]:
headers = {'Content-Type' : 'application/json; charset=utf-8'}
data = {
        "sender": "test_from",
        "recipient": "test_to",
        "amount": 3,
        "smart_contract": {"contract_address":"make_token2",
                           "contract_code" :"token_name = 'pyTOKEN' \ntoken_total_volume = 10000\ntoken_owner = {'USER1' : 10000}",
                           "contract_function_getBalance" :"""
def get_balance(user_id):
    print('{} Balance is : '.format(user_id), token_owner[user_id])
    return token_owner[user_id]
""",
                           "contract_function_sendToken" :"""
def send_token(sender,recipent,amount):
    if sender in token_owner.keys():
        if get_balance(sender) > amount:
            token_owner[sender]  = token_owner[sender] - amount
            if recipent in token_owner.keys():
                token_owner[recipent]  = token_owner[recipent] + amount
            else :
                token_owner[recipent]  =  amount
            print("Transaction Completed")
            get_balance(sender) 
            get_balance(recipent) 

        else:
            return "Insufficient Balance"
    else:
        return "Unavailable Sender id"
"""
                           

                                            }
            }
requests.post("http://localhost:5000/transactions/new", headers=headers, data=json.dumps(data)).content

headers = {'Content-Type' : 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/mine")
print(res)

<Response [200]>


In [18]:
headers = {'Content-Type' : 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/chain", headers=headers)
res_json = json.loads(res.content)

for _block in res_json['chain']:
    for _tx in _block['transactions']:
        if _tx['smart_contract']['contract_address'] == 'make_token2':
            exec( _tx['smart_contract']['contract_code'])
            break
    

In [19]:
token_name 

'pyTOKEN'

In [20]:
token_total_volume

10000

In [21]:
exec(_tx['smart_contract']['contract_function_getBalance'])
get_balance('USER1')

USER1 Balance is :  10000


10000

In [22]:
exec(_tx['smart_contract']['contract_function_sendToken'])
send_token('USER1','USER2',50)

USER1 Balance is :  10000
Transaction Completed
USER1 Balance is :  9950
USER2 Balance is :  50


In [23]:
exec(_tx['smart_contract']['contract_function_getBalance'])
get_balance('USER1')

USER1 Balance is :  9950


9950

In [25]:
exec(_tx['smart_contract']['contract_function_getBalance'])
get_balance('USER2')

USER2 Balance is :  50


50

In [ ]:
exec(_tx['smart_contract']['contract_function_sendToken'])
send_token('USER1','USER2',50)

In [ ]:
get_balance('USER1')

In [26]:
send_token('USER1','USER3',3000)

USER1 Balance is :  9950
Transaction Completed
USER1 Balance is :  6950
USER3 Balance is :  3000


In [27]:
exec(_tx['smart_contract']['contract_function_getBalance'])
get_balance('USER1')

USER1 Balance is :  6950


6950

In [28]:
exec(_tx['smart_contract']['contract_function_getBalance'])
get_balance('USER2')

USER2 Balance is :  50


50

In [29]:
exec(_tx['smart_contract']['contract_function_getBalance'])
get_balance('USER3')

USER3 Balance is :  3000


3000

In [ ]:
ccc = """token_name = "pyTOKEN" \
token_total_volume = 10000
                                            """

In [ ]:
exec(ccc)

In [ ]:
token_name

In [ ]:
for _block in res_json['chain']:
    for _tx in _block['transactions']:
        if _tx['smart_contract']['contract_address'] == 'myaddress':
            exec( _tx['smart_contract']['contract_code'])
            

In [ ]:
"x+y= {} + {}".format(3,3)

## Pandas 를 활용한 거래내역 조회

In [ ]:
status_json = json.loads(res.text)
status_json['chain']    
tx_amount_l = []
tx_sender_l = []
tx_reciv_l  = []
tx_time_l   = []

for chain_index in range(len(status_json['chain'])):
    chain_tx = status_json['chain'][chain_index]['transactions']
    for each_tx in range(len(chain_tx)):
        tx_amount_l.append(chain_tx[each_tx]['amount'])
        tx_sender_l.append(chain_tx[each_tx]['sender'])
        tx_reciv_l.append(chain_tx[each_tx]['recipient'])
        tx_time_l.append(chain_tx[each_tx]['timestamp'])

df_tx = pd.DataFrame()
df_tx['timestamp'] = tx_time_l  
df_tx['sender'] = tx_sender_l 
df_tx['recipient'] = tx_reciv_l
df_tx['amount'] = tx_amount_l   
df_tx

## 거래내역 기반 계정별 잔액 조회

In [ ]:
df_sended = pd.DataFrame(df_tx.groupby('sender')['amount'].sum()).reset_index()
df_sended.columns = ['user','sended_amount']
df_received= pd.DataFrame(df_tx.groupby('recipient')['amount'].sum()).reset_index()
df_received.columns = ['user','received_amount']
df_received

df_status = pd.merge(df_received,df_sended, on ='user', how=  'outer').fillna(0)
df_status['balance'] = df_status['received_amount']  - df_status['sended_amount']  
df_status